In [1]:
import sys
import os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

    dname = "conll003-englishversion"
    os.makedirs(dname, exist_ok=True)


    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=2686919c0cbcec16337651f9f42dbc9df315059feba1ce26fbd81731f097e875
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
class Args:
    data = "../input/conll003-englishversion"
    
args = Args()

In [4]:
def split_text_label(filename):
    f = open(filename)
    split_labeled_text = []
    sentence = []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                split_labeled_text.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        sentence.append([splits[0],splits[-1].rstrip("\n")])
    
    if len(sentence) > 0:
        split_labeled_text.append(sentence)
        sentence = []
    return split_labeled_text

In [5]:
split_train = split_text_label(os.path.join(args.data, "train.txt"))
split_valid = split_text_label(os.path.join(args.data, "valid.txt"))
split_test = split_text_label(os.path.join(args.data, "test.txt"))

In [6]:
split_train[:3]

[[['EU', 'B-ORG'],
  ['rejects', 'O'],
  ['German', 'B-MISC'],
  ['call', 'O'],
  ['to', 'O'],
  ['boycott', 'O'],
  ['British', 'B-MISC'],
  ['lamb', 'O'],
  ['.', 'O']],
 [['Peter', 'B-PER'], ['Blackburn', 'I-PER']],
 [['BRUSSELS', 'B-LOC'], ['1996-08-22', 'O']]]

In [7]:
split_valid[:3]

[[['CRICKET', 'O'],
  ['-', 'O'],
  ['LEICESTERSHIRE', 'B-ORG'],
  ['TAKE', 'O'],
  ['OVER', 'O'],
  ['AT', 'O'],
  ['TOP', 'O'],
  ['AFTER', 'O'],
  ['INNINGS', 'O'],
  ['VICTORY', 'O'],
  ['.', 'O']],
 [['LONDON', 'B-LOC'], ['1996-08-30', 'O']],
 [['West', 'B-MISC'],
  ['Indian', 'I-MISC'],
  ['all-rounder', 'O'],
  ['Phil', 'B-PER'],
  ['Simmons', 'I-PER'],
  ['took', 'O'],
  ['four', 'O'],
  ['for', 'O'],
  ['38', 'O'],
  ['on', 'O'],
  ['Friday', 'O'],
  ['as', 'O'],
  ['Leicestershire', 'B-ORG'],
  ['beat', 'O'],
  ['Somerset', 'B-ORG'],
  ['by', 'O'],
  ['an', 'O'],
  ['innings', 'O'],
  ['and', 'O'],
  ['39', 'O'],
  ['runs', 'O'],
  ['in', 'O'],
  ['two', 'O'],
  ['days', 'O'],
  ['to', 'O'],
  ['take', 'O'],
  ['over', 'O'],
  ['at', 'O'],
  ['the', 'O'],
  ['head', 'O'],
  ['of', 'O'],
  ['the', 'O'],
  ['county', 'O'],
  ['championship', 'O'],
  ['.', 'O']]]

In [8]:
split_test[:3]

[[['SOCCER', 'O'],
  ['-', 'O'],
  ['JAPAN', 'B-LOC'],
  ['GET', 'O'],
  ['LUCKY', 'O'],
  ['WIN', 'O'],
  [',', 'O'],
  ['CHINA', 'B-PER'],
  ['IN', 'O'],
  ['SURPRISE', 'O'],
  ['DEFEAT', 'O'],
  ['.', 'O']],
 [['Nadim', 'B-PER'], ['Ladki', 'I-PER']],
 [['AL-AIN', 'B-LOC'],
  [',', 'O'],
  ['United', 'B-LOC'],
  ['Arab', 'I-LOC'],
  ['Emirates', 'I-LOC'],
  ['1996-12-06', 'O']]]

In [9]:
labelSet = set()
wordSet = set()
# words and labels
for data in [split_train, split_valid, split_test]:
    for labeled_text in data:
        for word, label in labeled_text:
            labelSet.add(label)
            wordSet.add(word.lower())

In [10]:
print(labelSet)
#print(wordSet)

{'B-LOC', 'I-MISC', 'I-PER', 'B-PER', 'B-MISC', 'B-ORG', 'O', 'I-LOC', 'I-ORG'}


In [11]:
# Sort the set to ensure '0' is assigned to 0
sorted_labels = sorted(list(labelSet), key=len)

# Create mapping for labels
label2Idx = {}
for label in sorted_labels:
    label2Idx[label] = len(label2Idx)
idx2Label = {v: k for k, v in label2Idx.items()}

# Create mapping for words
word2Idx = {}
if len(word2Idx) == 0:
    word2Idx["PADDING_TOKEN"] = len(word2Idx)
    word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
for word in wordSet:
    word2Idx[word] = len(word2Idx)

In [12]:
def createMatrices(data, word2Idx, label2Idx):
    sentences = []
    labels = []
    for split_labeled_text in data:
        wordIndices = []
        labelIndices = []
        for word, label in split_labeled_text:
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]
            else:
                wordIdx = word2Idx['UNKNOWN_TOKEN']
            wordIndices.append(wordIdx)
            labelIndices.append(label2Idx[label])
        sentences.append(wordIndices)
        labels.append(labelIndices)
    return sentences, labels

In [13]:
train_sentences, train_labels = createMatrices(split_train, word2Idx, label2Idx)
valid_sentences, valid_labels = createMatrices(split_valid, word2Idx, label2Idx)
test_sentences, test_labels = createMatrices(split_test, word2Idx, label2Idx)

In [14]:
train_sentences[:3]

[[15307, 13600, 22869, 1742, 489, 4864, 1160, 10029, 17257],
 [9158, 21111],
 [10496, 15660]]

In [15]:
train_labels[:3]

[[4, 0, 8, 0, 0, 0, 8, 0, 0], [3, 2], [1, 0]]

In [ ]:
def padding(sentences, labels, max_en, padding='post')

In [ ]:
def padding(sentences, labels, max_len, padding='post'):
    padded_sentences = pad_sequences(sentences, max_len, padding='post')
    padded_labels = pad_sequences(labels, max_len, padding='post')
    return padded_sentences, padded_labels


In [ ]:
train_features, train_labels = padding(train_sentences, train_labels, max_seq_len, padding='post' )
valid_features, valid_labels = padding(valid_sentences, valid_labels, max_seq_len, padding='post' )
test_features, test_labels = padding(test_sentences, test_labels, max_seq_len, padding='post' )